# GPU内存

In [1]:
import torch

if torch.cuda.is_available():
    torch.cuda.empty_cache()

if torch.cuda.is_available():
    print(f"当前已分配显存: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"当前缓存中显存: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
else:
    print("无GPU")

GPU 内存已尝试释放。
当前已分配显存: 0.00 GB
当前缓存中显存: 0.00 GB


# 1 准备

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# 定义 max_memory 参数
# 键是设备编号 (0 代表第一张 GPU)，值是内存大小
# 你可以使用 "GiB", "MiB", "GB", "MB" 等单位
max_memory_map = {0: "15GiB"} 

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# 定义模型根目录
HF_CACHE_DIR = "/root/nas-private/huggingface_cache"
# 模型的相对路径（不带开头的 ./，保持路径结构清晰）
LLM_RELATIVE_PATH = "Qwen/Qwen1.5-7B-Chat" 
#LLM_RELATIVE_PATH = "Qwen/Qwen3-8b" 
EMBED_RELATIVE_PATH = "BAAI/bge-small-zh-v1.5" 
# 构造完整的本地路径
# 关键：确保这个路径是模型文件（如 config.json）所在文件夹的路径
LOCAL_LLM_PATH = os.path.join(HF_CACHE_DIR, LLM_RELATIVE_PATH)
LOCAL_EMBED_PATH =os.path.join(HF_CACHE_DIR,EMBED_RELATIVE_PATH)

print(LOCAL_LLM_PATH,LOCAL_EMBED_PATH)
tokenizer = AutoTokenizer.from_pretrained(LOCAL_LLM_PATH, trust_remote_code=True)
llm = HuggingFaceLLM(    # Qwen 1.5 default   context_window=3900, num_output=256
    #context_window=8192,  # 上下文窗口大小
    #max_new_tokens=2048,   # 最大生成长度
    tokenizer=tokenizer,
    model_name=LOCAL_LLM_PATH, # 注意这里改为本地路径
    model=AutoModelForCausalLM.from_pretrained(
        LOCAL_LLM_PATH,        # 注意这里改为本地路径
        device_map="auto",             # 自动分配到 GPU (3090)
       # torch_dtype="auto",  old
       dtype="auto",  #torch.float16,     # 使用 float16 减少显存占用
      #  load_in_4bit=True,             # 使用 4-bit 量化
        trust_remote_code=True,
        max_memory=max_memory_map
    )
)
# 初始化本地 Embedding Model
embed_model = HuggingFaceEmbedding(
    model_name=LOCAL_EMBED_PATH, # 注意这里改为本地路径
)
# 设置嵌入模型
Settings.embed_model = embed_model

# 设置LLM模型
Settings.llm =llm
print("ok")

/root/nas-private/huggingface_cache/Qwen/Qwen1.5-7B-Chat /root/nas-private/huggingface_cache/BAAI/bge-small-zh-v1.5


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ok


In [3]:
# 使用模型生成句子嵌入# 本练习不需要
from sentence_transformers import SentenceTransformer

sentences = ["This is an example sentence", "Each sentence is converted"]
model = SentenceTransformer(LOCAL_EMBED_PATH)

embeddings = model.encode(sentences)

In [4]:
len(embeddings),len(embeddings[0])

(2, 512)

# 2  chat

## 2.1 complete 模式

本模型不支持，因为是chat model

In [2]:
# 这种方法只适用于单次、无历史的提示
prompt = "写一个关于中国长城的小故事，不超过100字。"

# 使用 .complete() 方法（如果您的 HuggingFaceLLM 实例支持）
# 注意：通常 chat() 是推荐用于与 Chat 模型交互的方式，但 complete() 也可以用于简单的提示
try:
    response = Settings.llm.complete(prompt)
    print("--- LLM Response (Complete) ---")
    print(response.text)
    print("-------------------------------\n")
except AttributeError:
    print("Settings.llm 实例不支持 .complete() 方法，请使用包装后的 .chat()。")


--- LLM Response (Complete) ---

-------------------------------



## 2.2 chat 模式

### 2.2.1 一次

In [3]:
# 导入 ChatMessage，用于构造对话历史
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import Settings # 确保 Settings 在当前作用域可用

# 假设您想问一个关于中国的简单问题
prompt_content = "写一个关于中国长城的小故事，不超过100字。"

print(f"User Prompt: {prompt_content}\n")

# **关键修改：将字符串包装成 ChatMessage 列表**
messages = [
    ChatMessage(role=MessageRole.USER, content=prompt_content)
]

# 直接调用 llm 实例的 chat 方法，传入 messages 列表
response = Settings.llm.chat(messages)

print("--- LLM Response (Chat) ---")
print(response.message.content)
print("---------------------------\n")

User Prompt: 写一个关于中国长城的小故事，不超过100字。

--- LLM Response (Chat) ---
明朝年间，小明是个好奇的孩子。他听说那遥不可及的城墙能抵御外敌，就决定一探究竟。历经艰辛，他爬上长城，眼前一片雄伟。风中，他仿佛听到历史的回响，小小的心被长城的坚韧与勇气深深打动。从那以后，小明明白了，这不仅仅是一堵墙，它是中华儿女的骄傲。
---------------------------



### 2.2.2 多次

In [7]:
from llama_index.core.chat_engine import SimpleChatEngine

# 使用 Settings 中配置的 llm 创建 ChatEngine
chat_engine = SimpleChatEngine.from_defaults(
    llm=Settings.llm,
    # System Prompt 可选，用于设定模型角色
    system_prompt="你是一个乐于助人、充满好奇心的 AI 助手，你的回答总是简洁明了。",
)

print("--- Chat Engine Conversation ---")

# 第一次提问
response_1 = chat_engine.chat("给我推荐一个适合在秋天阅读的中文小说名字。")
print(f"Assistant: {response_1.response}\n")

# 第二次提问（ChatEngine 会自动维护历史）
response_2 = chat_engine.chat("这个作者还有其他的作品吗？")
print(f"Assistant: {response_2.response}\n")

print("--------------------------------\n")

--- Chat Engine Conversation ---
Assistant: 《围城》。这是钱钟书先生的经典之作，以讽刺和幽默的手法描绘了上世纪30年代中国社会的各种面貌，秋天读可以感受到季节的变迁，同时小说中的人生哲理也耐人寻味。

Assistant: 当然，钱钟书是中国现代文学的重要作家，除了《围城》，他的代表作还包括《管锥编》和《谈艺录》等，这些都是学术性和文化修养极高的著作。另外，他的妻子杨绛也有不少知名作品，如《洗澡》和《我们仨》等。

--------------------------------



## 3 工具调用测试

In [3]:
llm.metadata

LLMMetadata(context_window=8192, num_output=2048, is_chat_model=False, is_function_calling_model=False, model_name='/root/nas-private/huggingface_cache/Qwen3', system_role=<MessageRole.SYSTEM: 'system'>)

In [9]:
import asyncio
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings # 您的 LLM 在 Settings.llm 中
# ... (您的 LLM 和 Settings.llm 配置已完成) ...

# ----------------- 定义工具 (假设已成功) -----------------
def get_current_weather(city: str) -> str:
    """获取指定城市（例如：'北京'，'上海'）的当前天气信息。"""
    if "北京" in city:
        return "北京今天的气温是 28°C，多云转晴，空气质量良好。"
    elif "上海" in city:
        return "上海今天的气温是 -8°C，下雨，空气质量一般。"
    return f"抱歉，没有 {city} 的天气数据。"

def multiply(a: int, b: int) -> int:
    """将两个整数相乘。"""
    return a * b+100

weather_tool = FunctionTool.from_defaults(fn=get_current_weather)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
tools = [weather_tool, multiply_tool]


agent = ReActAgent(tools=tools, llm=Settings.llm, verbose=True)

# ----------------- 定义异步执行函数 -----------------
async def run_agent_queries(agent_instance: ReActAgent):
    print("--- 异步调用 1: 工具函数 (乘法) ---")
    agent_query_calc = "请计算一下 5 乘以 9 的结果是多少？"
    # 使用 .arun() 方法进行异步执行
    response_calc = await agent_instance.run(agent_query_calc)
    print(f"最终回答: {response_calc.response}\n")

    print("--- 异步调用 2: 工具函数 (天气) ---")
    agent_query_weather = "告诉我上海今天的天气怎么样？"
    response_weather = await agent_instance.run(agent_query_weather)
    print(f"最终回答: {response_weather.response}\n")
    
await run_agent_queries(agent)

--- 异步调用 1: 工具函数 (乘法) ---
最终回答: assistant: 5 乘以 9 的结果是 145。

--- 异步调用 2: 工具函数 (天气) ---
最终回答: assistant: 上海今天是-8°C，下雨，空气质量一般。记得出门带伞哦。



# Rag

In [10]:
documents = SimpleDirectoryReader("./txt").load_data()
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine()
response = query_engine.query("邓国标工作单位是什么，复旦教育经历的是什么")
print(response)

邓国标的工作单位是XTransfer（上海夺畅网络技术有限公司），担任CEO兼联合创始人。他的复旦教育经历是在1998年参加的上海市高级管理干部培训班（百人工程）第五期，专业是高端培训。


In [14]:
nodes=index.as_query_engine(llm=Settings.llm).query("用中文回答，施小琳教育经历")

In [21]:
print(nodes)

Empty Response


In [17]:
len(nodes.source_nodes)

2

In [19]:
nodes.source_nodes[0].text

'以下内容与施小琳有关\n姓名：施小琳\n性别：女\n婚姻状况\t其他\n居住地\t上海市\n更新时间\t2023/03/01\n施小琳标签\t两会代表, 院长级, 二十大人大代表, 第十四届全国人大代表（2023）\n施小琳项目\t政府培训班\n\n施小琳教育经历：复旦管院政府培训班/1998上海市高级管理干部培训班(百人工程)第五期/高端培训\t\n\n默认邮箱\tshi_xiaolin@hotmail.com\n\n施小琳当前工作(工作单位/部门/职位)：四川省省委/副书记;四川省政府/党组书记;四川省人民政府/省长\n\n行业\t政府机构/非营利机构;政府机构/非营利机构;政府机构/非营利机构\n\n二类行业\t政府机构/非营利机构;政府机构/非营利机构;政府机构/非营利机构\n\n---\n\n施小琳\n工作履历(工作单位/部门/职位)\t上海市普陀区人民政府/区委书记、区委委员;上海市民政局/局长;南汇区人民政府/副区长;虹口区人民政府/副区长;上海市委常委/统战部部长;江西省委/常委;江西省宣传部/部长;四川省成都市/成都市委书记、兼任成都警备区党委第一书记;四川省人民政府/副省长、代理省长\n\n政府类职级\t省部级副职;省部级副职;省部级副职;省部级副职;省部级正职\n\n人际关系(是否校友/姓名/关系/公司/职位/手机号/邮箱/项目班级)\t非校友/陈叶丹/无/无/无/13818353646/无/无\n\n# 施小琳职务变更\n## \t2024/07/31 施小琳任四川省 省长\t成都发布\t\n\n## \t2024/07/04 施小琳任四川省副省长、代理省长\t\t澎湃新闻\t\n\n## \t施小琳已任四川省政府党组书记\t2024/06/29\t澎湃新闻\t\n\n## \t成都市委书记施小琳履新四川省委副书记\t2023/07/03\t人民网\n\n# 人大代表\n中国共产党第二十次全国代表大会代表名单\t2022/10/16\t新华社\n\n---\n\n施小琳有关新闻\n\n新华社成都2024年7月31日电\u2003四川省第十四届人民代表大会第三次会议7月31日选举施小琳为四川省人民政府省长。'

In [22]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\n only Using"
    " the context information, not using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you will not "
    " answer the question on your own.\n 所有回答以中文显示 \n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing the new"
    " context , update or repeat the existing answer.\n 所有回答以中文显示 \n"
)
refine_template = PromptTemplate(refine_template_str)

In [23]:

    index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=llm,
    ).query("按时间顺序，列出施小琳的所有职务变更，如果有时间也列出来。")
)

Empty Response
